In [1]:
%%capture
!pip install plotly --upgrade
!pip install "jupyterlab>=3" "ipywidgets>=7.6"
get_ipython().kernel.do_shutdown(True)

In [2]:
%%capture
!pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 

In [3]:
!jupyter labextension install jupyterlab-plotly@5.7.0 @jupyter-widgets/jupyterlab-manager

-\|/-\|/-\|/-\|/-\|/-\|/-\|/--\|/-\|/-\|/-\|/-Building jupyterlab assets (production, minimized)
-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-

In [1]:
import plotly
plotly.__version__

'5.10.0'

In [2]:
%%capture
!pip install https://github.com/OlivierBeq/Papyrus-scripts/tarball/master
!pip install -U kaleido
get_ipython().kernel.do_shutdown(True)

In [1]:
!gdown "https://drive.google.com/uc?id=1hrSb8ouyD9F47ndiuMXlK-VmovrDThnV&confirm=t"
!gdown "https://drive.google.com/uc?id=15zbhi5S8e-xFe-V3KJhqPPaMCz00dVrV&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1hrSb8ouyD9F47ndiuMXlK-VmovrDThnV&confirm=t
To: /content/05.4_combined_set_protein_targets.tsv.xz
100% 1.70M/1.70M [00:00<00:00, 30.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=15zbhi5S8e-xFe-V3KJhqPPaMCz00dVrV&confirm=t
To: /content/05.5_combined_set_without_stereochemistry.tsv.xz
100% 719M/719M [00:06<00:00, 105MB/s]


In [6]:
!gdown "https://drive.google.com/uc?id=1A7XDndNywq_g6g0gdy3LR8PGvjXgYf_Y&confirm=t"
!gdown "https://drive.google.com/uc?id=15zbhi5S8e-xFe-V3KJhqPPaMCz00dVrV&confirm=t"
!gdown "https://drive.google.com/uc?id=1s4ShxWrTGRFa8DRkrpKw-4N-x-JRnlrI&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1A7XDndNywq_g6g0gdy3LR8PGvjXgYf_Y&confirm=t
To: /content/05.5_combined_set_protein_targets.tsv.xz
100% 1.71M/1.71M [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=15zbhi5S8e-xFe-V3KJhqPPaMCz00dVrV&confirm=t
To: /content/05.5_combined_set_without_stereochemistry.tsv.xz
100% 719M/719M [00:06<00:00, 113MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s4ShxWrTGRFa8DRkrpKw-4N-x-JRnlrI&confirm=t
To: /content/05.5_combined_3D_set_with_stereochemistry.sd.xz
100% 492M/492M [00:04<00:00, 108MB/s] 


In [7]:
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from papyrus_scripts.reader import read_protein_set

In [8]:
protein_data = pd.read_csv('05.5_combined_set_protein_targets.tsv.xz', sep='\t', keep_default_na=False)

In [11]:
classes = defaultdict(dict)
root = ' '
classes[root] = {'parent': None, 'value': 0}
for chunk in tqdm(pd.read_csv('05.5_combined_set_without_stereochemistry.tsv.xz',
                              sep='\t', chunksize=1000000, usecols=['target_id']),
                  total=60):
    tmp = chunk.merge(protein_data, on='target_id')[['Classification']]
    tmp = tmp.Classification.str.split(';', expand=True).iloc[:, 0].str.split('->', expand=True)
    tmp.columns = [f'level{i}' for i in range(1, len(tmp.columns) + 1)]
    for i, row in tqdm(tmp.iterrows(), total=tmp.shape[0], leave=False):
        if len(row['level1']) == 0:
            classes[root]['value'] += 1
            continue
        classes[root]['value'] += 1
        for i_level in range(tmp.shape[1]):
            if i_level == 0:
                if row['level1'] not in classes.keys():
                    classes[row['level1']] = {'parent': root, 'value': 1}
                else:
                    classes[row['level1']]['value'] += 1
            elif row[f'level{i_level + 1}'] is not None:
                if row[f'level{i_level + 1}'] not in classes.keys():
                    classes[row[f'level{i_level + 1}']] = {'parent': row[f'level{i_level}'], 'value': 1}
                else:
                    classes[row[f'level{i_level + 1}']]['value'] += 1

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

...

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/775087 [00:00<?, ?it/s]

In [12]:
import pickle
with open('protein_classes_05.5_fixed.pkl', 'wb') as oh:
    pickle.dump(classes, oh)
from google.colab import files
files.download('protein_classes_05.5_fixed.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle
with open('protein_classes_05.5.pkl', 'rb') as oh:
    classes = pickle.load(oh)

In [13]:
node, parent, value = zip(*[(key, value['parent'], value['value']) for key, value in classes.items()])

In [14]:
import plotly.graph_objects as go
import plotly.express as px
import textwrap

In [15]:
def customwrap(s,width=15):
    return "<br>".join(textwrap.wrap(s,width=width))

In [16]:
node = list(map(customwrap, node))
parent = [parent[0]] + list(map(customwrap, parent[1:]))
node[0] = ' '
parent[1:] = [x if len(x) else node[0] for x in parent[1:]]

In [17]:
layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=0,
    )
)

In [18]:
sunburst = go.Sunburst(labels=node, parents=parent, values=value,
                    #    maxdepth=3, 
                       insidetextorientation='horizontal', #'radial',
                    #    branchvalues='total',
                       branchvalues='remainder',
                    outsidetextfont = {"size": 20, "color": "#000000", 'family': 'Times New Roman'},
                    insidetextfont = {"size": 20, "color": "#000000", 'family': 'Times New Roman'},
                    )

In [19]:
fig = go.Figure(data=[sunburst], layout=layout)
# fig.update_layout(uniformtext=dict(minsize=20, mode='show'))\
#    .update_traces(texttemplate='%{label}<br>%{value}<br>%{percentEntry:.2%}',
#                   textfont=dict(size=[20 for _ in range(len(node))])
                #  )
fig.update_traces(texttemplate='%{label}<br>%{value}',
                #   textfont=dict(size=[20 for _ in range(len(node))])
                  )
fig.show()
# fig.write_image('sunburst_5.5.svg')
fig.write_html('sunburst_all_levels_05.5_fixed.html', include_plotlyjs=True)
from google.colab import files
files.download('sunburst_all_levels_05.5_fixed.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>